In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import dspy

if hasattr(dspy, 'cache') and hasattr(dspy.cache, 'disk_cache'):
    dspy.cache.disk_cache.clear()
    print("clear")

clear


In [3]:
class ChatSignature(dspy.Signature):
    """Respond to user in a conversation"""
    conversation_history: str = dspy.InputField(desc="Previous messages")
    user_message: str = dspy.InputField(desc="Current user input")
    response: str = dspy.OutputField(desc="Assistant's response")

class Chatbot:
    def __init__(self):
        self.predictor = dspy.Predict(ChatSignature)
        self.history = []
    
    def chat(self, user_message: str) -> str:
        # Format history
        history_str = "\n".join([f"{m['role']}: {m['content']}" for m in self.history])
        
        # Get response using signature
        result = self.predictor(conversation_history=history_str, user_message=user_message)
        
        # Update history
        self.history.append({"role": "user", "content": user_message})
        self.history.append({"role": "assistant", "content": result.response})
        
        return result.response


In [9]:
from package.base import DriverLM, ModelResponse, Usage
import httpx
from typing import Any

ollama_client = httpx.Client(timeout=600.0)

def ollama_request_fn(messages:list[dict[str, Any]], temperature:float=0.0, **kwargs)->dict:
    base_url = 'http://localhost:11434/api/chat'

    response = ollama_client.post(
        base_url,
        json={
            "model": "llama3.2-vision:11b",
            "messages": messages,
            "stream": False,
            "options": {"temperature": temperature}
        }
    )
    response.raise_for_status()
    return response.json()  # Return full Ollama response

def ollama_output_fn(response:dict)->ModelResponse:
    content = response.get("message", {}).get("content", "")
    model = response.get("model", "custom")
    
    usage = Usage(
        prompt_tokens=response.get("prompt_eval_count", 0),
        completion_tokens=response.get("eval_count", 0),
        total_tokens=response.get("prompt_eval_count", 0) + response.get("eval_count", 0)
    )
    
    return ModelResponse.from_text(text=content.strip(), usage=usage, model=model)

In [10]:
response = ollama_request_fn(messages=[{"role": "user", "content": "Hi"}])
ollama_output_fn(response)

ModelResponse(id='chatcmpl-1e974956-1cfe-45e5-9aae-7ece836e85e1', created=1768728070, model='llama3.2-vision:11b', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='How can I assist you?', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None, reasoning_content=None))], usage={'prompt_tokens': 10, 'completion_tokens': 7, 'total_tokens': 17}, cache_hit=False)

In [11]:
ollama_lm = DriverLM(
    request_fn=ollama_request_fn,
    output_fn=ollama_output_fn,
    cache=True
)

ollama_lm.clear_cache()  # Clear old cache entries

dspy.configure(lm=ollama_lm)

In [38]:
class ChatSignature(dspy.Signature):
    """Response to user in a conversation"""
    chat_history:str = dspy.InputField(desc="Previous chat history")
    user_message:str = dspy.InputField(desc="Current user message")
    response:str = dspy.OutputField(desc="Assistant's resposne")

class ChatAgent(dspy.Module):
    def __init__(self):
        self.program = dspy.Predict(ChatSignature)
        self.history = []

    def forward(self, message:str):
        if len(self.history)>0:
            chat_history = "\n".join([f"{m['role'].upper()}: {m['content']}" for m in self.history])
        else:
            chat_history = "No previous messages"
        response = self.program(chat_history=chat_history, user_message=message)
        self.history.append({"role": "user", "content": message})
        self.history.append({"role": "assistant", "content": response.response})
        return response.response


In [39]:
dspy.configure(lm=ollama_lm)
ca = ChatAgent()

In [40]:
ca.forward(message="My name is Bank. Nice to meet you.")

2026/01/18 16:32:59 WARNING dspy.primitives.module: Calling module.forward(...) on ChatAgent directly is discouraged. Please use module(...) instead.


'Nice to meet you too, Bank! What brings you here today?'

In [34]:
ca(message="What's my name?")

'Your name is Bank.'

In [41]:
print(ollama_lm.history[-1]['messages'][0]['content'])

Your input fields are:
1. `chat_history` (str): Previous chat history
2. `user_message` (str): Current user message
Your output fields are:
1. `response` (str): Assistant's resposne
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## chat_history ## ]]
{chat_history}

[[ ## user_message ## ]]
{user_message}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Response to user in a conversation


In [42]:
print(ollama_lm.history[-1]['messages'][-1]['content'])

[[ ## chat_history ## ]]
No previous messages

[[ ## user_message ## ]]
My name is Bank. Nice to meet you.

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


In [43]:
ollama_lm.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': "Your input fields are:\n1. `chat_history` (str): Previous chat history\n2. `user_message` (str): Current user message\nYour output fields are:\n1. `response` (str): Assistant's resposne\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## chat_history ## ]]\n{chat_history}\n\n[[ ## user_message ## ]]\n{user_message}\n\n[[ ## response ## ]]\n{response}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Response to user in a conversation"},
  {'role': 'user',
   'content': '[[ ## chat_history ## ]]\nNo previous messages\n\n[[ ## user_message ## ]]\nMy name is Bank. Nice to meet you.\n\nRespond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.'}],
 'kwargs': {},
 'response': ModelResponse(id='chatcmpl-ed42d1e8-52a0-4873-aca6-c3042f779dc5', cr